# Import Statements

In [1]:
import gurobipy as gp
import networkx as nx
from networkx.algorithms.approximation.steinertree import steiner_tree as steiner_tree_2
import random
from math import log
import scipy
from scipy.optimize import newton
from scipy.optimize import root_scalar
import sympy

# Constants

## Debug Level Flags

In [2]:
DEBUG_LEVEL_NONE = 0
DEBUG_LEVEL_THEORY_1 = 1.1
DEBUG_LEVEL_THEORY_2 = 1.2
DEBUG_LEVEL_FULL = 2
DEBUG_LEVEL_EXTREME = 3

## General Constants

In [3]:
DEBUG_LEVEL = DEBUG_LEVEL_THEORY_1
NUM_NODES = 100
NUM_EDGES = 150
DEGREE = 3
NUM_MULTICAST_REQUESTS = 50
MAX_MULTICAST_SIZE = NUM_NODES/5
TOLERANCE = 0.1

NATURAL = 0
UNIFORM = 1
PERTURB = 2

# Generating Problem Instances

In [4]:
def is_connected(G):
    return nx.is_k_edge_connected(G, 1)

def get_random_graph():
    G = nx.empty_graph(NUM_NODES)
    while not is_connected(G):
        G = nx.random_regular_graph(DEGREE, NUM_NODES)
    return G

def get_random_multicast_request(G):
    k = random.randint(2, MAX_MULTICAST_SIZE)
    multicast_group = random.sample(list(G), k)
    source = multicast_group[0]
    recipients = set(multicast_group[1:])
    return source, recipients

def get_random_instance():
    G = get_random_graph()
    multicast_requests = list()
    for i in range(NUM_MULTICAST_REQUESTS):
        multicast_requests.append(get_random_multicast_request(G))
    return G, multicast_requests

# Jansen-Zhang General Convex min-max

In [5]:
# Don't know if I need this right now
# Define non-negative convex function f:B-R^M
# def define_f(multicast_instance):
#    k = len(multicast_instance[1])
#    G = multicast_instance[0]
#    def f(x, e):
#        retval = 0
#        for i in range(k):
#            retval += sum(x[i])
#        return retval
#    return f      

# Define a function that computes theta_t(x)
def theta_via_sympy(x, t, f, G, lambda_of_x):
    M = len(G.edges())
    theta = sympy.Symbol('theta')
    expression = 1
    for e in G.edges():
        expression -= (t/M) * theta/(theta - f(x,e))
    soln_set = sympy.solvers.solveset(expression, theta, domain=sympy.Interval(lambda_of_x, sympy.S.Infinity, True, True))
    #assert len(soln_set == 1)
    if DEBUG_LEVEL >= DEBUG_LEVEL_FULL:
        print(soln_set)
    for soln in soln_set:
        retval = soln
    return retval

def theta_eq(theta, x, t, f, G, lambda_of_x):
    M = len(G.edges())
    retval = 1
    for e in G.edges():
        retval = retval - t*theta/(M*(theta - f(x,e)))
    print("theta = {}".format(theta))
    print("theta_eq = {}".format(retval))
    return retval

def derivative_theta_eq(theta, x, t, f, G, lambda_of_x):
    M = len(G.edges())
    retval = 0
    for e in G.edges():
        retval = retval + t*f(x,e)/(M*(theta - f(x,e))*(theta - f(x,e)))
    print("d_theta_eq = {}".format(retval))
    return retval

# Define a function that computes p_t_e(x)
def jz_price(x, e, t, f, G, lambda_of_x):
    M = len(G.edges())
    theta_of_x = scipy.optimize.newton(theta_eq, x0, args=(x, t, f, G, lambda_of_x))
    return t*theta_of_x/(M*(theta_of_x-f(x,e)))

# Create (Reduced) LP

In [6]:
def create_LP(G, multicast_requests):
    reduced_LP = gp.Model("Multicast Packing Model - Reduced")
    congestion = reduced_LP.addVar(name="lambda")
    reduced_LP.setObjective(congestion, gp.GRB.MINIMIZE)
    reduced_LP.update()
    # Packing Constraints
    for e in G.edges():
        reduced_LP.addConstr(congestion >= 0, name="{} congestion".format(sorted(e)))
    # Simplex Constraints
    for i in range(len(multicast_requests)):
        reduced_LP.addConstr(0*congestion == 1, name="Tree Selection for {}".format(i))
    reduced_LP.update()
    return reduced_LP

# Column Generating Subproblem

In [7]:
def generate_new_trees(G, multicast_requests, prices):
    nx.set_edge_attributes(G, prices, "weight")
    assert nx.is_weighted(G)
    new_trees = list()
    for request in multicast_requests:
        new_trees.append(steiner_tree_2(G, request[1].union([request[0]])))
    return new_trees

def add_trees_to_LP(G, multicast_requests, reduced_LP, new_trees):
    assert len(multicast_requests) == len(new_trees)
    for i in range(len(new_trees)):
        coeffs = list()
        constrs = list()
        # the new tree contributes to the constraint for each of its edges
        for e in new_trees[i].edges():
            coeffs.append(-1)
            constrs.append(reduced_LP.getConstrByName("{} congestion".format(sorted(e))))
        coeffs.append(1)
        # the new tree contributes to the selection constraint for the corresponding multicast request
        constrs.append(reduced_LP.getConstrByName("Tree Selection for {}".format(i)))
        reduced_LP.addVar(name="x{}{}".format(i, nx.info(new_trees[i])), column=gp.Column(coeffs, constrs))
        reduced_LP.update()
        
def cost(G):
    assert nx.is_weighted(G)
    prices = nx.get_edge_attributes(G, "weight")
    return sum([prices[e] for e in G.edges()])

# Main Solver

In [8]:
def solve_multicast_packing_via_columngen(G, multicast_requests, priceFlag=NATURAL):
    reduced_LP = create_LP(G, multicast_requests)
    new_trees = generate_new_trees(G, multicast_requests, 1/len(G.edges()))
    stop_flag = False
    iteration = 0
    x = []
    lambdas = []
    thetas = []
    prices = {}
    uniform_prices = {}
    perturbed_prices = {}
    t=TOLERANCE
    multicast_costs = [None] * len(multicast_requests)
    
    while not stop_flag:
        add_trees_to_LP(G, multicast_requests, reduced_LP, new_trees)
        reduced_LP.optimize()
        x.append(reduced_LP.getVars())
        
        if DEBUG_LEVEL >= DEBUG_LEVEL_EXTREME:
            print(reduced_LP.getA().toarray())
            print(reduced_LP.getAttr("RHS"))
        
        lambda_of_x = reduced_LP.getObjective().getValue()
        lambdas.append(lambda_of_x)
        f = lambda x, e: lambda_of_x + reduced_LP.getConstrByName("{} congestion".format(sorted(e))).getAttr(gp.GRB.Attr.Slack) # Note: Gurobi signs their slacks stupidly
        #solution = root_scalar(theta_eq, x0=(1+t)*lambda_of_x, args=(x, t, f, G, lambda_of_x), fprime=derivative_theta_eq)
        #print(solution)
        #theta_of_x = solution.root
        theta_of_x = lambda_of_x/(1-t)
        thetas.append(theta_of_x)
        
        for e in G.edges():
            prices[e] = reduced_LP.getConstrByName("{} congestion".format(sorted(e))).getAttr(gp.GRB.Attr.Pi)
            if prices[e] > 0:
                uniform_prices[e] = 1
            else:
                uniform_prices[e] = 0
            perturbed_prices[e] = t*theta_of_x/(len(G.edges())*(theta_of_x-f(x,e)))
            
        if priceFlag == NATURAL:
            new_trees = generate_new_trees(G, multicast_requests, prices)
        if priceFlag == UNIFORM:
            new_trees = generate_new_trees(G, multicast_requests, uniform_prices)
        if priceFlag == PERTURB:
            new_trees = generate_new_trees(G, multicast_requests, perturbed_prices)
        
        if False:
            new_trees_uniform = generate_new_trees(G, multicast_requests, uniform_prices)
            new_trees_perturbed = generate_new_trees(G, multicast_requests, perturbed_prices)
            newVsUniform = [not any(new_trees[i].edges() - new_trees_uniform[i].edges()) for i in range(len(multicast_requests))]
            newVsPerturbed = [not any(new_trees[i].edges() - new_trees_perturbed[i].edges()) for i in range(len(multicast_requests))]
            UniformVsPerturbed = [not any(new_trees_uniform[i].edges() - new_trees_perturbed[i].edges()) for i in range(len(multicast_requests))]
            print("New Tree = New Trees with Uniform Prices: {}".format(all(newVsUniform)))
            print("New Tree = New Trees with Uniform Prices: {}".format(all(newVsPerturbed)))
            print("New Tree = New Trees with Uniform Prices: {}".format(all(UniformVsPerturbed)))
        
        for i in range(len(multicast_requests)):
            multicast_costs[i] = reduced_LP.getConstrByName("Tree Selection for {}".format(i)).getAttr(gp.GRB.Attr.Pi)
        
        if all([cost(new_trees[i]) - multicast_costs[i] >= 0 for i in range(len(multicast_requests))]):
            stop_flag = True
        if sum([cost(new_trees[i]) for i in range(len(multicast_requests))]) >= lambda_of_x:
            stop_flag = True
        
        if DEBUG_LEVEL >= DEBUG_LEVEL_THEORY_1:
            print(iteration)
            print("lambda(x): {}".format(lambda_of_x))
            print("log lambda(x): {}".format(log(lambda_of_x)))
            #print("theta_{}(x): {}".format(TOLERANCE, theta(x, TOLERANCE, f, G, lambda_of_x)))
            print("theta_{}(x): {}".format(TOLERANCE, theta_of_x))
        if DEBUG_LEVEL >= DEBUG_LEVEL_THEORY_2:
            for e in G.edges():
                if prices[e] > 0:
                    t = sympy.Symbol('t')
                    p = sympy.limit_seq(jz_price(x, t, f, G, lambda_of_x), t)
                    p = jz_price(x, e, TOLERANCE, f, G, lambda_of_x)
                    print("{}: price: {} \t JZ price: {} \t slack: {}".format(e, prices[e], p, reduced_LP.getConstrByName("{} congestion".format(sorted(e))).getAttr(gp.GRB.Attr.Slack)))
            
        iteration += 1
        

# Testing

In [9]:
def test_instance_generation(num_tests):
    assert not is_connected(nx.empty_graph(NUM_NODES))
    assert is_connected(get_random_graph())
    G = get_random_graph()
    for i in range(num_tests):
        source, recipients = get_random_multicast_request(G)
        assert source != None
        assert recipients
        assert not source in recipients
        assert isinstance(recipients,set)
    G, multicast_requests = get_random_instance()
    for request in multicast_requests:
        source = request[0]
        recipients = request[1]
        assert source in G
        assert recipients.issubset(G)

def test_LP_creation(num_tests):
    reduced_LP = create_LP(*get_random_instance())
    assert len(reduced_LP.getVars()) == 1
    assert len(reduced_LP.getConstrs()) == NUM_EDGES + NUM_MULTICAST_REQUESTS, "Num of constraints: {} Expected: {}".format(len(reduced_LP.getConstrs()), NUM_EDGES + NUM_MULTICAST_REQUESTS)
    
def test_subproblem(num_tests):
    G, multicast_requests = get_random_instance()
    reduced_LP = create_LP(G, multicast_requests)
    
    new_trees = generate_new_trees(G, multicast_requests, 1)
    assert (len(new_trees) == len(multicast_requests))
    for i in range(len(new_trees)):
        #assert new_trees[i] == steiner_tree_2(G, multicast_requests[i][1].union([multicast_requests[i][0]]), 1)
        assert(cost(new_trees[i]) == len(new_trees[i].edges()))
           
    add_trees_to_LP(G, multicast_requests, reduced_LP, new_trees)
    assert(len(reduced_LP.getVars()) == 1+len(multicast_requests))
    
def run_tests(num_tests):
    test_instance_generation(num_tests)
    test_LP_creation(num_tests)
    test_subproblem(num_tests)

NUM_TESTS = 100
run_tests(NUM_TESTS)

# Run the solver

In [10]:
for i in range(1):
    print("BEGIN NEW TEST")
    instance = get_random_instance()
    print("Natural Prices")
    solve_multicast_packing_via_columngen(*instance, priceFlag=NATURAL)
    print("Uniform Prices")
    solve_multicast_packing_via_columngen(*instance, priceFlag=UNIFORM)
    #print("Perturbed Prices")
    #solve_multicast_packing_via_columngen(*instance, priceFlag=PERTURB)

BEGIN NEW TEST
Natural Prices
0
lambda(x): 16.0
log lambda(x): 2.772588722239781
theta_0.1(x): 17.77777777777778
1
lambda(x): 13.0
log lambda(x): 2.5649493574615367
theta_0.1(x): 14.444444444444445
2
lambda(x): 11.3125
log lambda(x): 2.4259083090260445
theta_0.1(x): 12.569444444444445
3
lambda(x): 9.895633116854
log lambda(x): 2.2920935605199504
theta_0.1(x): 10.995147907615555
4
lambda(x): 9.361612132131468
log lambda(x): 2.2366175119973573
theta_0.1(x): 10.401791257923852
5
lambda(x): 9.14958223807975
log lambda(x): 2.213708221204651
theta_0.1(x): 10.166202486755278
6
lambda(x): 8.993447512068252
log lambda(x): 2.1964962579614027
theta_0.1(x): 9.992719457853614
7
lambda(x): 8.863616700574557
log lambda(x): 2.1819548868062535
theta_0.1(x): 9.848463000638397
8
lambda(x): 8.766676686466571
log lambda(x): 2.170957793457911
theta_0.1(x): 9.740751873851746
9
lambda(x): 8.671225751198541
log lambda(x): 2.160010159253731
theta_0.1(x): 9.63469527910949
10
lambda(x): 8.59888127745001
log lambd